In [23]:
from openai import OpenAI
from sentence_transformers import SentenceTransformer, util
from tqdm import tqdm
import time
import numpy as np
import json
from itertools import combinations_with_replacement
import json
from tqdm import tqdm
import argparse

In [25]:
model = SentenceTransformer('multi-qa-MiniLM-L6-cos-v1')

prompt = '"Can you suggest some books similar to \"To Kill a Mockingbird\"?"'

advertisers = ['Velora', 'BookHaven', 'MassMart', 'EspressoEdge']
ads = [
    "Discover the future of technology with Velora, the brand that redefines innovation and elegance. Velora designs and sells a premium range of smartphones, tablets, laptops, and smartwatches, all crafted to seamlessly integrate into your lifestyle. Our products are engineered with user-friendly interfaces, stunning designs, and cutting-edge technology to keep you connected and productive. Velora's ecosystem offers unparalleled synchronization across devices, ensuring a smooth and efficient experience whether you're at work, school, or on the go. With Velora Pay, you can enjoy secure and convenient payment services, while our robust cloud service keeps your data safe and accessible anytime, anywhere. Elevate your tech experience with Velora, where sophistication meets simplicity and advanced functionality.",
    "Introducing BookHaven, your ultimate online bookstore where the world of literature is just a click away. At BookHaven, we offer an extensive collection of books spanning every genre and interest, from timeless classics and gripping thrillers to insightful non-fiction and enchanting children's stories. Our user-friendly platform ensures a seamless shopping experience, with personalized recommendations and unbeatable prices. Whether you're a voracious reader or just looking for your next great read, BookHaven is dedicated to delivering literary treasures right to your doorstep with fast, reliable shipping and a hassle-free return policy. Discover the joy of reading with BookHaven, where every book finds its perfect reader. Dive into a world of endless possibilities and let your next adventure begin at BookHaven!",
    "Experience the joy of shopping at MassMart, where quality meets value in a dynamic retail environment tailored for your satisfaction. At MassMart, members enjoy exclusive access to a vast selection of premium, bulk-sized products, from fresh groceries to high-tech electronics, all at unbeatably low prices. With a commitment to customer happiness, sustainability, and community support, MassMart isn't just a shopping destination — it's a part of your community. Dive into a world of savings and discover why millions choose MassMart as their trusted shopping partner. Join us today and see the difference MassMart can make in your shopping experience, where every visit is more than just shopping — it's an adventure!",
    "Experience the warmth and delight of EspressoEdge, where every sip offers an invitation to a world of exquisite flavors and aromas. Renowned globally for its high-quality, handcrafted beverages, EspressoEdge is committed to sourcing the finest Arabica beans, expertly blending them into a variety of rich espressos, frothy cappuccinos, and creamy lattes. Each visit to an EspressoEdge store is more than just a coffee run—it's an opportunity to savor a moment of luxury amid the hustle of daily life. Whether you seek the comfort of a familiar classic or the thrill of a new seasonal specialty, EspressoEdge welcomes all to gather, connect, and enjoy a cup perfectly tailored to your taste. Step into your local EspressoEdge today and join us in celebrating the art of coffee.",
]


bids = np.array([3, 3, 2, 2,]) # Scenario 0

# advertisers = ['Velora', 'BookHaven', 'MassMart', 'EspressoEdge', 'SocialHub', 'ColaBubbles', 'FizzyPop', 'SkyTech', 'AeroDynamics', 'MusicStream', 'BrainChips']
# ads = [
#     "Discover the future of technology with Velora, the brand that redefines innovation and elegance. Velora designs and sells a premium range of smartphones, tablets, laptops, and smartwatches, all crafted to seamlessly integrate into your lifestyle. Our products are engineered with user-friendly interfaces, stunning designs, and cutting-edge technology to keep you connected and productive. Velora's ecosystem offers unparalleled synchronization across devices, ensuring a smooth and efficient experience whether you're at work, school, or on the go. With Velora Pay, you can enjoy secure and convenient payment services, while our robust cloud service keeps your data safe and accessible anytime, anywhere. Elevate your tech experience with Velora, where sophistication meets simplicity and advanced functionality.",
#     "Introducing BookHaven, your ultimate online bookstore where the world of literature is just a click away. At BookHaven, we offer an extensive collection of books spanning every genre and interest, from timeless classics and gripping thrillers to insightful non-fiction and enchanting children's stories. Our user-friendly platform ensures a seamless shopping experience, with personalized recommendations and unbeatable prices. Whether you're a voracious reader or just looking for your next great read, BookHaven is dedicated to delivering literary treasures right to your doorstep with fast, reliable shipping and a hassle-free return policy. Discover the joy of reading with BookHaven, where every book finds its perfect reader. Dive into a world of endless possibilities and let your next adventure begin at BookHaven!",
#     "Experience the joy of shopping at MassMart, where quality meets value in a dynamic retail environment tailored for your satisfaction. At MassMart, members enjoy exclusive access to a vast selection of premium, bulk-sized products, from fresh groceries to high-tech electronics, all at unbeatably low prices. With a commitment to customer happiness, sustainability, and community support, MassMart isn't just a shopping destination — it's a part of your community. Dive into a world of savings and discover why millions choose MassMart as their trusted shopping partner. Join us today and see the difference MassMart can make in your shopping experience, where every visit is more than just shopping — it's an adventure!",
#     "Experience the warmth and delight of EspressoEdge, where every sip offers an invitation to a world of exquisite flavors and aromas. Renowned globally for its high-quality, handcrafted beverages, EspressoEdge is committed to sourcing the finest Arabica beans, expertly blending them into a variety of rich espressos, frothy cappuccinos, and creamy lattes. Each visit to an EspressoEdge store is more than just a coffee run—it's an opportunity to savor a moment of luxury amid the hustle of daily life. Whether you seek the comfort of a familiar classic or the thrill of a new seasonal specialty, EspressoEdge welcomes all to gather, connect, and enjoy a cup perfectly tailored to your taste. Step into your local EspressoEdge today and join us in celebrating the art of coffee.",
#     "Discover the power of connection with SocialHub, the world's leading social media platform. With over two billion active users, SocialHub is your gateway to staying in touch with friends and family, discovering new communities, and sharing your life's moments. Our innovative features, from personalized news feeds to interactive groups, make it easy to engage with what matters most to you. Whether you're promoting your business, staying updated on the latest news, or simply keeping up with loved ones, SocialHub is the ultimate tool to enhance your digital experience. Join us today and be part of a global network where connections come to life!",
#     "Experience the refreshing taste of ColaBubbles, the world's favorite soft drink. With its unique blend of flavors and effervescent bubbles, ColaBubbles has been bringing joy to people of all ages for over a century. Whether you're enjoying a moment of relaxation, celebrating with friends, or on the go, ColaBubbles is the perfect companion to quench your thirst and uplift your spirits. Our commitment to quality and tradition ensures every sip is as delightful as the first. Indulge in the classic taste of ColaBubbles and make every moment special. Taste the feeling!",
#     "Unleash the bold taste of FizzyPop, the iconic soft drink that invigorates and refreshes like no other. Known for its crisp, refreshing flavor and vibrant effervescence, FizzyPop is the perfect choice for those who dare to live life to the fullest. Whether you're at a party, watching a game, or simply taking a break, FizzyPop brings a burst of excitement to any occasion. With a heritage of quality and a commitment to innovation, every sip of FizzyPop delivers an unmatched experience. Embrace the bold, and make every moment extraordinary with the unmistakable taste of FizzyPop.",
#     "Explore the skies with SkyTech, the world's leading aerospace company renowned for its innovation, quality, and reliability. SkyTech designs, manufactures, and services commercial airplanes, defense systems, and space technologies, making global connectivity and exploration possible. Whether you're traveling for business or leisure, SkyTech's state-of-the-art aircraft ensure a safe, comfortable, and efficient journey. With a legacy of pioneering advancements and a commitment to excellence, SkyTech continues to shape the future of aviation. Choose SkyTech and experience the pinnacle of aerospace engineering and performance. Fly with confidence, fly with SkyTech.",
#     "Experience the future of aviation with AeroDynamics, the global leader in aerospace innovation and excellence. AeroDynamics designs and manufactures the world’s most advanced commercial aircraft, providing unparalleled comfort, efficiency, and reliability. From cutting-edge technology to sustainable solutions, AeroDynamics is dedicated to shaping the future of air travel. Whether you're embarking on a long-haul journey or a short domestic flight, AeroDynamics ensures a superior flying experience with spacious cabins, innovative features, and top-notch safety standards. Trust AeroDynamics for a seamless and enjoyable journey every time. Fly smarter, fly with AeroDynamics.",
#     "Immerse yourself in the world of music with MusicStream, the ultimate destination for streaming your favorite tunes anytime, anywhere. With a vast library of millions of songs, playlists curated just for you, and personalized recommendations, MusicStream puts the power of music discovery in your hands. Whether you're in the mood for chart-topping hits, underground gems, or soothing melodies, MusicStream has something for everyone. Plus, with offline listening capabilities and seamless integration across devices, you can take your music with you wherever you go. Join the millions of music lovers worldwide and unlock endless possibilities with MusicStream. Discover, stream, and experience the joy of music like never before.", 
#     "Experience the cutting-edge innovation of BrainChips, the global leader in semiconductor technology. BrainChips' groundbreaking processors power the devices that fuel our modern world, from laptops and desktops to servers and cloud computing systems. With a legacy of pushing the boundaries of technology, BrainChips continues to deliver industry-leading performance, reliability, and security. Whether you're a professional tackling complex tasks or a gamer seeking immersive experiences, BrainChips processors provide the power and efficiency you need. Trust BrainChips to deliver the performance you demand and the reliability you can count on. Join the millions who rely on BrainChips technology and unlock new possibilities for productivity, creativity, and entertainment.",
# ]



# # bids = np.array([3, 3, 2, 2,]) # Scenario 0
# bids = np.array([2, 1, 3, 3,]) # Scenario 1
# bids = np.array([1, 4, 1, 1, 2, 2, 2,]) # Scenario 2
# bids = np.array([1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]) # Scenario 3

prompt_for_adv = 'Give me a paragraph about {} so that I can use it to advertise this brand.'

In [27]:
methods = [
    'single_alloc_with_replacement',
    'single_alloc_without_replacement',
    'single_alloc_naive_i',
    'single_alloc_naive_ii',
    'multi_alloc_vcg',
]

In [28]:
SOCIAL_WELFARE = 'social_welfare'
REVENUE = 'revenue'
INDIVIDUAL_WELFARE = 'individual_welfare'
RELEVANCE = 'relevance'
OUTPUT = 'output'
WINNER = 'winner'
AUCTION_SCORES = 'auction_scores'
RELEVANCE_TO_ORIGINAL = 'relevance_to_original'
SEGMENT_RELEVANCE_TO_ORIGINAL = "segment_relevence_to_original"
OUTPUT_MODIFIED = 'output_modified'

### Generate Original Answers for Scenario 0

In [10]:
def query_to_chatgpt(prompt):
    chat_completion_det = client.chat.completions.create(
        messages = [
            {
                "role": "user",
                "content": prompt
            },
            {
                "role": "assistant",
                "content": ""
            }
        ],
        model = "gpt-4-turbo",
        logprobs = False,
        temperature = 1,
        max_tokens = 300
    )
    return chat_completion_det.choices[0].message.content

In [11]:
query = f'''
{prompt}
Answer this question for only 1 sentence.
'''

In [12]:
query_to_chatgpt(query)

'Sure! You might enjoy "The Help" by Kathryn Stockett, which explores themes of race and social justice in the South, much like "To Kill a Mockingbird." Another recommendation is "A Time to Kill" by John Grisham, which also tackles issues of racism and legal ethics in the Southern United States.'

In [12]:
orig_y = []
for i in range(10):
    orig_y.append(query_to_chatgpt(query))
    time.sleep(0.5)

In [13]:
with open('scenario0_output_1/orig_y.json', 'w') as f:
    f.write(json.dumps(orig_y, indent=4))

In [29]:
with open('scenario0_output_1/orig_y.json', 'r') as f:
    orig_y = json.loads(f.read())

In [30]:
def relevance_to_original_output(original_output: list, outputs_with_ad: list):
    orig_embedding = model.encode(original_output)
    ads_embedding = model.encode(outputs_with_ad)
    return (1 + util.dot_score(ads_embedding, orig_embedding, ).numpy()) / 2

In [31]:
def add_relevance_to_output_to_metrics(path_file: str, use_output_modified: bool=False):
    try:
        with open(path_file, 'r') as f:
            all_metrics = json.loads(f.read())
    except:
        all_metrics = []
        
    print(f'number of runs: {len(all_metrics)}')

    for metric in tqdm(all_metrics):
        if use_output_modified:
            outputs = metric[OUTPUT_MODIFIED]
        else:
            outputs = metric[OUTPUT]
        
        rels = relevance_to_original_output(original_output=orig_y, outputs_with_ad=outputs)
        rels = np.mean(rels, axis=1)
        rels = [float(x) for x in rels]
        
        metric[RELEVANCE_TO_ORIGINAL] = rels
    
    
    st = []
    for metric in all_metrics:
        st.append(metric[RELEVANCE_TO_ORIGINAL])

    st = np.array(st)
    result = np.mean(st, axis=0)
    
    with open(path_file, 'w') as f:
        f.write(json.dumps(all_metrics, indent=4))
    
    return [f'k={i} {result[i]:.3f}' for i in range(result.shape[0])], result, st
    # return f'k=1: {result[0]:.3f}\nk=2: {result[1]:.3f}\nk=3: {result[2]:.3f}', result


In [16]:
def fix_outputs_naive_i(path_file: str):
    try:
        with open(path_file, 'r') as f:
            all_metrics = json.loads(f.read())
    except:
        all_metrics = []
    
    print(f'number of runs: {len(all_metrics)}')
    
    sample_starts = []
    for i in range(10):
        sample_starts.append(query_to_chatgpt(query))
    
    for metric in tqdm(all_metrics):
        sample_id = np.random.randint(0, 10)
        sample_start = sample_starts[sample_id]
        metric[OUTPUT_MODIFIED] = [sample_start + metric[OUTPUT][i] for i in range(len(metric[OUTPUT]))]
    
    with open(path_file, 'w') as f:
        f.write(json.dumps(all_metrics, indent=4))
        

In [17]:
method_id = 2
fix_outputs_naive_i(f'scenario0_output_1/{methods[method_id]}_dep{0}.json')

number of runs: 100


100%|██████████| 100/100 [00:00<00:00, 111343.35it/s]


In [38]:
all_res = []

In [39]:
scenario = 0

### Average of Relevance to Original Output

When having $k=1$, $k=2$, and $k=3$ advertisement segments.

#### Single Allocation with Replcement

In [40]:
method_id = 0
out, t, s = add_relevance_to_output_to_metrics(f'scenario{scenario}_output_1/{methods[method_id]}_dep{0}.json')
print(out)
all_res.append(s)

number of runs: 100


100%|██████████| 100/100 [00:02<00:00, 47.98it/s]

['k=0 0.746', 'k=1 0.715', 'k=2 0.700']


#### Single Allocation without Replcement

In [41]:
method_id = 1
out, t, s = add_relevance_to_output_to_metrics(f'scenario{scenario}_output_1/{methods[method_id]}_dep{0}.json')
print(out)
all_res.append(s)

number of runs: 100


100%|██████████| 100/100 [00:02<00:00, 44.73it/s]

['k=0 0.752', 'k=1 0.716', 'k=2 0.702']


In [15]:
def rag_based_relevance(x: str, ads: list): 
    x_embedding = model.encode(x)
    ads_embedding = model.encode(ads)
    bf = (1 + util.dot_score(x_embedding, ads_embedding)[0].numpy()) / 2
    return (bf - 0.4) /0.35

def get_allocation_vector(prompt: str, bids: np.ndarray, ads: list, advertisers: list):
    q = rag_based_relevance(prompt, ads, )
    x = q * bids / np.sum(q * bids)
    
    output = { i: { 'q': float(q[i]), 'bid': float(bids[i]), 'adv': advertisers[i], 'x': float(x[i])} for i in range(len(advertisers))}
    
    print(f'(q_i, b_i, adv_i)')
    for y in list(zip(q, bids, advertisers)):
        print(f'({y[0]:.2f}, {y[1]:.2f}, {y[2]})')
        
    print(f'allocation vector:\n{x}')
    print(f'x_max / x_min: {np.max(x) / np.min(x)}')

    return output

alloc_output = get_allocation_vector(prompt, bids, ads, advertisers)

(q_i, b_i, adv_i)
(0.36, 1.00, Velora)
(0.87, 1.00, BookHaven)
(0.31, 1.00, MassMart)
(0.26, 1.00, EspressoEdge)
(0.21, 1.00, SocialHub)
(0.36, 1.00, ColaBubbles)
(0.38, 1.00, FizzyPop)
(0.28, 1.00, SkyTech)
(0.33, 1.00, AeroDynamics)
(0.34, 1.00, MusicStream)
(0.33, 1.00, BrainChips)
allocation vector:
[0.08853651 0.21578052 0.07629691 0.06459284 0.05317406 0.0880366
 0.09470903 0.07033001 0.08184636 0.08430283 0.08239433]
x_max / x_min: 4.058003763758804


#### Single Allocation Naive (i)

without adding original answer to the beginning.

In [43]:
method_id = 2
out, t, s = add_relevance_to_output_to_metrics(f'scenario{scenario}_output_1/{methods[method_id]}_dep{0}.json')
print(out)
all_res.append(s)

number of runs: 100


100%|██████████| 100/100 [00:02<00:00, 38.42it/s]


['k=0 0.559', 'k=1 0.570', 'k=2 0.567']


when adding original answer to the beginning.

In [44]:
method_id = 2
out, t, s = add_relevance_to_output_to_metrics(f'scenario{scenario}_output_1/{methods[method_id]}_dep{0}.json', use_output_modified=True)
print(out)
all_res.append(s)

number of runs: 100


100%|██████████| 100/100 [00:02<00:00, 39.95it/s]

['k=0 0.743', 'k=1 0.740', 'k=2 0.671']


#### Single Allocation Naive (ii)

In [45]:
method_id = 3
out, t, s = add_relevance_to_output_to_metrics(f'scenario{scenario}_output_1/{methods[method_id]}_dep{0}.json')
print(out)
all_res.append(s)

number of runs: 100


100%|██████████| 100/100 [00:02<00:00, 48.40it/s]


['k=0 0.745', 'k=1 0.712', 'k=2 0.698']


In [46]:
all_res = np.stack(all_res)

In [47]:
all_res.shape

(5, 100, 3)

In [25]:
import pandas as pd
df = pd.DataFrame({
    'Mechanism': ['with Replacement', 'without Replacement', 'Naive (i) (wo output)', 'Naive (i) (w output)', 'Naive (ii)'],
    'k=1': [f'{x:.3f}' for x in all_res[:, 0]],
    'k=2': [f'{x:.3f}' for x in all_res[:, 1]],
    'k=3': [f'{x:.3f}' for x in all_res[:, 2]]
})
df

,Mechanism,k=1,k=2,k=3
0,with Replacement,0.746,0.715,0.700
1,without Replacement,0.752,0.716,0.702
2,Naive (i) (wo output),0.559,0.570,0.567
3,Naive (i) (w output),0.743,0.740,0.671
4,Naive (ii),0.745,0.712,0.698


In [50]:
import pandas as pd
df = pd.DataFrame({
    'Mechanism': ['with Replacement', 'without Replacement', 'Naive (i) (wo output)', 'Naive (i) (w output)', 'Naive (ii)'],
    'k=1': [f'{np.mean(x):.3f} ($\pm$ {(np.std(x)/x.shape[0]**0.5):.4f})' for x in all_res[:, :, 0]],
    'k=2': [f'{np.mean(x):.3f} ($\pm$ {(np.std(x)/x.shape[0]**0.5):.4f})' for x in all_res[:, :, 1]],
    'k=3': [f'{np.mean(x):.3f} ($\pm$ {(np.std(x)/x.shape[0]**0.5):.4f})' for x in all_res[:, :, 2]],
})
df

,Mechanism,k=1,k=2,k=3
0,with Replacement,0.746 ($\pm$ 0.0040),0.715 ($\pm$ 0.0039),0.700 ($\pm$ 0.0036)
1,without Replacement,0.752 ($\pm$ 0.0040),0.716 ($\pm$ 0.0035),0.702 ($\pm$ 0.0034)
2,Naive (i) (wo output),0.559 ($\pm$ 0.0032),0.570 ($\pm$ 0.0029),0.567 ($\pm$ 0.0025)
3,Naive (i) (w output),0.743 ($\pm$ 0.0043),0.740 ($\pm$ 0.0044),0.671 ($\pm$ 0.0032)
4,Naive (ii),0.745 ($\pm$ 0.0048),0.712 ($\pm$ 0.0045),0.698 ($\pm$ 0.0040)


### Segment-wise Relevency

In this part, in order to get the similarity of the whole output to the original outputs, we measure the relevancy of each individual segment to those and take the average.

In [51]:
def segment_wise_relevancy(path_file: str, use_output_modified: bool=False):
    try:
        with open(path_file, 'r') as f:
            all_metrics = json.loads(f.read())
    except:
        all_metrics = []
        
    print(f'number of runs: {len(all_metrics)}')

    for metric in tqdm(all_metrics):
        if use_output_modified:
            outputs = metric[OUTPUT_MODIFIED]
        else:
            outputs = metric[OUTPUT]
        
        segments = []
        for k in range(3):
            if k == 0:
                segments.append(outputs[0])
            else:
                prv_len = len(outputs[k - 1])
                segments.append(outputs[k][prv_len:])
        
        
        rels = relevance_to_original_output(original_output=orig_y, outputs_with_ad=segments)
        rels = np.mean(rels, axis=1)
        rels = [float(x) for x in rels]
        
        metric[SEGMENT_RELEVANCE_TO_ORIGINAL] = rels
    
    
    st = []
    for metric in all_metrics:
        st.append(metric[SEGMENT_RELEVANCE_TO_ORIGINAL])

    st = np.array(st)
    result = np.mean(st, axis=0)
    
    with open(path_file, 'w') as f:
        f.write(json.dumps(all_metrics, indent=4))
        
    return f'k=1: {result[0]:.3f}\nk=2: {result[1]:.3f}\nk=3: {result[2]:.3f}', result, st


In [65]:
segments_res = []

In [66]:
method_id = 0
out, t, s = segment_wise_relevancy(f'scenario{scenario}_output_1/{methods[method_id]}_dep{0}.json')
print(out)
segments_res.append(s)

number of runs: 100


100%|██████████| 100/100 [00:01<00:00, 51.83it/s]

k=1: 0.746
k=2: 0.596
k=3: 0.588


In [67]:
method_id = 1
out, t, s = segment_wise_relevancy(f'scenario{scenario}_output_1/{methods[method_id]}_dep{0}.json')
print(out)
segments_res.append(s)

number of runs: 100


100%|██████████| 100/100 [00:01<00:00, 53.48it/s]


k=1: 0.752
k=2: 0.602
k=3: 0.576


In [68]:
method_id = 2
out, t, s = segment_wise_relevancy(f'scenario{scenario}_output_1/{methods[method_id]}_dep{0}.json')
print(out)
segments_res.append(s)

number of runs: 100


100%|██████████| 100/100 [00:01<00:00, 50.87it/s]

k=1: 0.559
k=2: 0.555
k=3: 0.551


In [69]:
method_id = 2
out, t, s = segment_wise_relevancy(f'scenario{scenario}_output_1/{methods[method_id]}_dep{0}.json', use_output_modified=True)
print(out)
segments_res.append(s)

number of runs: 100


100%|██████████| 100/100 [00:01<00:00, 51.07it/s]


k=1: 0.743
k=2: 0.555
k=3: 0.551


In [70]:
method_id = 3
out, t, s = segment_wise_relevancy(f'scenario{scenario}_output_1/{methods[method_id]}_dep{0}.json')
print(out)
segments_res.append(s)

number of runs: 100


100%|██████████| 100/100 [00:01<00:00, 53.08it/s]


k=1: 0.745
k=2: 0.600
k=3: 0.584


In [71]:
np_segments_res = np.stack(segments_res)

In [72]:
np_segments_res.shape

(5, 100, 3)

In [34]:
import pandas as pd
df = pd.DataFrame({
    'Mechanism': ['with Replacement', 'without Replacement', 'Naive (i) (wo output)', 'Naive (i) (w output)', 'Naive (ii)'],
    'k=1': [f'{x:.3f}' for x in np_segments_res[:, 0]],
    'k=2': [f'{x:.3f}' for x in np_segments_res[:, 1]],
    'k=3': [f'{x:.3f}' for x in np_segments_res[:, 2]]
})
df

,Mechanism,k=1,k=2,k=3
0,with Replacement,0.746,0.596,0.588
1,without Replacement,0.752,0.602,0.576
2,Naive (i) (wo output),0.559,0.555,0.551
3,Naive (i) (w output),0.743,0.555,0.551
4,Naive (ii),0.745,0.600,0.584


In [73]:
import pandas as pd
df = pd.DataFrame({
    'Mechanism': ['with Replacement', 'without Replacement', 'Naive (i) (wo output)', 'Naive (i) (w output)', 'Naive (ii)'],
    'k=1': [f'{np.mean(x):.3f} ($\pm$ {(np.std(x)/x.shape[0]**0.5):.4f})' for x in np_segments_res[:, :, 0]],
    'k=2': [f'{np.mean(x):.3f} ($\pm$ {(np.std(x)/x.shape[0]**0.5):.4f})' for x in np_segments_res[:, :, 1]],
    'k=3': [f'{np.mean(x):.3f} ($\pm$ {(np.std(x)/x.shape[0]**0.5):.4f})' for x in np_segments_res[:, :, 2]],
})
df

,Mechanism,k=1,k=2,k=3
0,with Replacement,0.746 ($\pm$ 0.0040),0.596 ($\pm$ 0.0040),0.588 ($\pm$ 0.0039)
1,without Replacement,0.752 ($\pm$ 0.0040),0.602 ($\pm$ 0.0045),0.576 ($\pm$ 0.0043)
2,Naive (i) (wo output),0.559 ($\pm$ 0.0032),0.555 ($\pm$ 0.0033),0.551 ($\pm$ 0.0035)
3,Naive (i) (w output),0.743 ($\pm$ 0.0043),0.555 ($\pm$ 0.0033),0.551 ($\pm$ 0.0035)
4,Naive (ii),0.745 ($\pm$ 0.0048),0.600 ($\pm$ 0.0040),0.584 ($\pm$ 0.0047)


In [35]:
import pandas as pd
df = pd.DataFrame({
    'Mechanism': ['with Replacement', 'without Replacement', 'Naive (i) (wo output)', 'Naive (i) (w output)', 'Naive (ii)'],
    'k<=1': [f'{x:.3f}' for x in np_segments_res[:, 0]],
    'k<=2': [f'{x:.3f}' for x in np.mean(np_segments_res[:, :2], axis=1)],
    'k<=3': [f'{x:.3f}' for x in np.mean(np_segments_res[:, :3], axis=1)],
})
df

,Mechanism,k<=1,k<=2,k<=3
0,with Replacement,0.746,0.671,0.643
1,without Replacement,0.752,0.677,0.643
2,Naive (i) (wo output),0.559,0.557,0.555
3,Naive (i) (w output),0.743,0.649,0.616
4,Naive (ii),0.745,0.673,0.643


In [36]:
import pandas as pd
df = pd.DataFrame({
    'Mechanism': ['with Replacement', 'without Replacement', 'Naive (i) (wo output)', 'Naive (i) (w output)', 'Naive (ii)'],
    'Average over k=[1, 2, 3]': [f'{x:.3f}' for x in np.mean(np_segments_res, axis=1)],
})
df

,Mechanism,"Average over k=[1, 2, 3]"
0,with Replacement,0.643
1,without Replacement,0.643
2,Naive (i) (wo output),0.555
3,Naive (i) (w output),0.616
4,Naive (ii),0.643
